In [1]:
from pyspark.sql import SparkSession

# Creation de la session Spark 
spark = SparkSession.builder \
    .appName("Log server Analysis") \
    .master("local[*]") \
    .getOrCreate()


# Creation  du Context Spark 
sc = spark.sparkContext

# afficher les informations sur le contexte spark
print("Spark Context Version:", sc.version)
print("Application Name:", sc.appName)

Spark Context Version: 3.5.4
Application Name: Log server Analysis


In [2]:
# Lire le fichier 
path = "Dataset - Projet 3 copy.txt"

df = spark.read.text(path)
df.show()

+--------------------+
|               value|
+--------------------+
|Last login: Fri J...|
|serge-nd@MacBook-...|
|PING 8.8.8.8 (8.8...|
|64 bytes from 8.8...|
|64 bytes from 8.8...|
|64 bytes from 8.8...|
|64 bytes from 8.8...|
|64 bytes from 8.8...|
|64 bytes from 8.8...|
|64 bytes from 8.8...|
|64 bytes from 8.8...|
|64 bytes from 8.8...|
|64 bytes from 8.8...|
|64 bytes from 8.8...|
|64 bytes from 8.8...|
|64 bytes from 8.8...|
|64 bytes from 8.8...|
|64 bytes from 8.8...|
|64 bytes from 8.8...|
|64 bytes from 8.8...|
+--------------------+
only showing top 20 rows



In [3]:
type(df)

pyspark.sql.dataframe.DataFrame

In [4]:
new_rdd=df.rdd.zipWithIndex().filter(lambda x: x[1] > 2).map(lambda x: x[0])
type(new_rdd)

pyspark.rdd.PipelinedRDD

In [5]:
nr_row = new_rdd.count()
print( "Number of rows ", nr_row)

Number of rows  80222


In [6]:
first_row = new_rdd.first()
print( "first row ", first_row)

first row  Row(value='64 bytes from 8.8.8.8: icmp_seq=0 ttl=105 time=548.917 ms')


In [7]:
new_rdd.take(10)

[Row(value='64 bytes from 8.8.8.8: icmp_seq=0 ttl=105 time=548.917 ms'),
 Row(value='64 bytes from 8.8.8.8: icmp_seq=1 ttl=105 time=350.260 ms'),
 Row(value='64 bytes from 8.8.8.8: icmp_seq=2 ttl=105 time=530.040 ms'),
 Row(value='64 bytes from 8.8.8.8: icmp_seq=3 ttl=105 time=370.254 ms'),
 Row(value='64 bytes from 8.8.8.8: icmp_seq=4 ttl=105 time=424.650 ms'),
 Row(value='64 bytes from 8.8.8.8: icmp_seq=5 ttl=105 time=142.222 ms'),
 Row(value='64 bytes from 8.8.8.8: icmp_seq=6 ttl=105 time=364.631 ms'),
 Row(value='64 bytes from 8.8.8.8: icmp_seq=7 ttl=105 time=303.672 ms'),
 Row(value='64 bytes from 8.8.8.8: icmp_seq=8 ttl=105 time=315.577 ms'),
 Row(value='64 bytes from 8.8.8.8: icmp_seq=9 ttl=105 time=258.308 ms')]

In [14]:
len(new_rdd.take(1)[0])

1

In [15]:
import re

# Définir la fonction pour extraire les valeurs pertinentes
def extraire_valeurs(row):
    # Expressions régulières pour correspondre aux champs requis icmp_seq[= ](\d+)  
    match_bytes = re.search(r"(\d+) bytes", row['value'])
    match_icmp_seq = re.search(r"icmp_seq[= ](\d+)", row['value'])
    match_ttl = re.search(r"ttl=(\d+)", row['value'])
    match_time = re.search(r"time=(\d+\.?\d*)", row['value'])
    
    # Si tous les éléments sont trouvés, retourner les valeurs extraites
    try :
        return [
            int(match_bytes.group(1)) if match_bytes else None,    # bytes
            int(match_icmp_seq.group(1)) if match_icmp_seq else None, # icmp_seq
            int(match_ttl.group(1)) if match_ttl else None,      # ttl
            float(match_time.group(1)) if match_time else None    # time
        ]
    except AttributeError:  # Gère le cas où .group() est appelé sur None
        return None  # Si la ligne ne correspond pas au modèle, retourner None

# Appliquer la transformation pour extraire les champs requis de chaque ligne de l'RDD
rdd_extrait = new_rdd.map(lambda row: extraire_valeurs(row))

# Afficher les 10 premiers éléments extraits
#rdd_extrait.take(10)


In [16]:
print("Nombre de requetes  avant extraction des informations:", new_rdd.count())
print("Nombre  total de requetes apres extraction des informations:", rdd_extrait.count())
print("Nombre  total de requetes envoyées:", rdd_extrait.filter(lambda x: x[1] is not None).count())
print("nombre total de réponses recues:", rdd_extrait.filter(lambda x: x[3] is not None).count())
print("Nombre de requete pour lesquelles il n'ya pas de route pour le host:", rdd_extrait.filter(lambda x: x[1] is  None).count())    #le système ne trouve pas de route réseau valide pour atteindre l'hôte cible.

Nombre de requetes  avant extraction des informations: 80222
Nombre  total de requetes apres extraction des informations: 80222
Nombre  total de requetes envoyées: 76529
nombre total de réponses recues: 65722
Nombre de requete pour lesquelles il n'ya pas de route pour le host: 3693


### **II.2. Indicateurs de performance réseau**

##### **1. Temps maximal**

In [18]:
rdd_received= rdd_extrait.filter(lambda x: x[3] is not None)
temps_maximal = rdd_received.map(lambda x: x[3]).max()
print(f"Temps maximal : {temps_maximal:,} ms")

Temps maximal : 927,118.712 ms


##### **2. Temps minimal**

In [19]:
temps_minimal = rdd_received.map(lambda x: x[3]).min()
print(f"Temps minimal : {temps_minimal} ms")

Temps minimal : 35.597 ms


##### **3. Temps moyen**

In [20]:
temps_moyen = rdd_received.map(lambda x: x[3]).mean()
print(f"Temps moyen : {temps_moyen:.3f} ms")

Temps moyen : 634.931 ms


In [21]:
import numpy as np
latences_rdd = rdd_extrait.filter(lambda x: x[3] is not None)
latences=latences_rdd.map(lambda x: x[3]).collect()

print(f"Le premier quartile est: {np.percentile(latences, 25)} ms")
print(f"Le deuxième quartile est: {np.percentile(latences, 50)} ms")
print(f"Le troisieme quartile est: {np.percentile(latences, 75)} ms")

Le premier quartile est: 131.07 ms
Le deuxième quartile est: 133.824 ms
Le troisieme quartile est: 140.17875 ms


##### **4. Variabilité de la latence ( ecart type, coefficient de variation)**

In [22]:
ecart_type = rdd_received.map(lambda x: x[3]).stdev()
print(f"Écart type du temps : {ecart_type:,.3f} ms")
print(f"coefficient de variation : {100*(ecart_type/temps_moyen):,.3f}%")

Écart type du temps : 15,968.798 ms
coefficient de variation : 2,515.046%


##### **5. Taux de perte de paquets**

In [ ]:
# Calcul du taux de perte de paquets
received= rdd_extrait.filter(lambda x: x[3] is not None).count()
total = rdd_extrait.count()
taux_1 = ((total-received)/total) * 100
print(f"Taux de perte de paquets : {taux_1:.3f}%")


Taux de perte de paquets : 18.075%


##### **6. Les différents time to live**

In [23]:
ttl_uniques = rdd_received.map(lambda x: x[2]).distinct()
print("TTL uniques:", ttl_uniques.collect())

TTL uniques: [100, 114, 110, 105, 103, 109]


##### **7. statistique sur les time to live**

In [24]:
# Calcul des statistiques sur les time to live (TTL)
ttl_maximal = rdd_received.map(lambda x: x[2]).max()
ttl_minimal = rdd_received.map(lambda x: x[2]).min()
ttl_moyen = rdd_received.map(lambda x: x[2]).mean()
ttl_ecart_type = rdd_received.map(lambda x: x[2]).stdev()

print(f"TTL maximal : {ttl_maximal}")
print(f"TTL minimal : {ttl_minimal}")
print(f"TTL moyen : {ttl_moyen:.3f}")
print(f"Écart type du TTL : {ttl_ecart_type:.3f}")
print(f"Coefficient de variation du TTL : {100*(ttl_ecart_type/ttl_moyen):.3f}%")


TTL maximal : 114
TTL minimal : 100
TTL moyen : 112.533
Écart type du TTL : 3.118
Coefficient de variation du TTL : 2.771%


<font style="font-family: Centaur; font-size: 22px;">
Les statistiques du  TTL  indiquent une  bonne stabilité du réseau :  

TTL moyen élevé (112.533) signifie qu'il ya un faible nombre de routeurs traversés.  
Faible écart-type (3.118) et CV (2.771%) donc  les paquets suivent des routes cohérentes.  
Plage de TTL (100-114) modérée  d'ou les variations restent maîtrisées.  </font>


##### **8. Latence moyenne par intervalle de 10 secondes**

###### **Regroupement en groupe d'intervalle de 10**

In [25]:
rdd_indexed = rdd_extrait.zipWithIndex()  # Associe chaque élément à son index global

# Regroupement par intervalle de 10 secondes
rdd_grouped = rdd_indexed.map(lambda x: (x[1] // 10, x[0])) \
                         .groupByKey() \
                         .mapValues(list) \
                         .sortByKey() 


In [26]:
rdd_grouped.take(2)

[(0,
  [[64, 0, 105, 548.917],
   [64, 1, 105, 350.26],
   [64, 2, 105, 530.04],
   [64, 3, 105, 370.254],
   [64, 4, 105, 424.65],
   [64, 5, 105, 142.222],
   [64, 6, 105, 364.631],
   [64, 7, 105, 303.672],
   [64, 8, 105, 315.577],
   [64, 9, 105, 258.308]]),
 (1,
  [[64, 10, 105, 656.045],
   [64, 11, 105, 377.271],
   [64, 12, 105, 702.369],
   [64, 13, 105, 402.506],
   [64, 14, 105, 537.176],
   [64, 15, 105, 452.802],
   [64, 16, 105, 481.603],
   [64, 17, 105, 335.816],
   [64, 18, 105, 636.983],
   [64, 19, 105, 257.781]])]

###### **Comptage du nombre de groupe**

In [27]:
rdd_grouped.count()

8023

In [28]:
def moyenne_par_groupe(rdd_grouped):
    def moyenne(groupe):
        valeurs = [x[3] for x in groupe if  x[3] is not None]
        somme = sum(valeurs)
        nombre = len(valeurs)
        return f"{(somme / nombre):.2f}" if nombre > 0 else None

    return rdd_grouped.map(lambda groupe: (groupe[0], moyenne(groupe[1])))

# Utilisation de la fonction
moyenne_temps_par_groupe = moyenne_par_groupe(rdd_grouped)
moyenne_temps_par_groupe.take(10)


[(0, '360.85'),
 (1, '484.04'),
 (2, '507.97'),
 (3, '620.92'),
 (4, '546.34'),
 (5, '540.22'),
 (6, '588.29'),
 (7, '718.23'),
 (8, '610.84'),
 (9, '859.39')]

##### **9.  anomalies où le temps de latence dépasse un certain seuil**

###### **a.  par exemple, 500 ms**

In [29]:
seuil_latence = 500
anomalies = rdd_extrait.filter(lambda x: x[3] is not None and x[3] > seuil_latence)
print(f"le nombre de requetes ayant abouties et dont le temps de latence à dépassé {seuil_latence}ms est: ",anomalies.count())
print("les ttl correspondant sont :", anomalies.map(lambda x: x[2]).distinct().collect())
ttl_rdd =anomalies
ttl_counts = ttl_rdd.map(lambda x: x[2]).countByValue()

print("Nombre d'occurrences pour chaque TTL:")
for ttl, count in ttl_counts.items():
    print(f"TTL {ttl}: {count}")


le nombre de requetes ayant abouties et dont le temps de latence à dépassé 500ms est:  3257
les ttl correspondant sont : [100, 114, 110, 105, 103, 109]
Nombre d'occurrences pour chaque TTL:
TTL 105: 2927
TTL 100: 5
TTL 103: 71
TTL 114: 151
TTL 110: 5
TTL 109: 98


###### **b.  par exemple, 5000 ms**

In [30]:
seuil_latence = 5000
anomalies = rdd_extrait.filter(lambda x: x[3] is not None and x[3] > seuil_latence)
print(f"le nombre de requetes ayant abouties et dont le temps de latence à dépassé {seuil_latence}ms est: ",anomalies.count())
print("les ttl correspondant sont :", anomalies.map(lambda x: x[2]).distinct().collect())
ttl_rdd =anomalies
ttl_counts = ttl_rdd.map(lambda x: x[2]).countByValue()

print("Nombre d'occurrences pour chaque TTL:")
for ttl, count in ttl_counts.items():
    print(f"TTL {ttl}: {count}")

le nombre de requetes ayant abouties et dont le temps de latence à dépassé 5000ms est:  1144
les ttl correspondant sont : [114, 105, 109]
Nombre d'occurrences pour chaque TTL:
TTL 105: 1100
TTL 114: 33
TTL 109: 11


###### **c.  par exemple, 15 000 ms**

In [31]:
seuil_latence = 15000
anomalies = rdd_extrait.filter(lambda x: x[3] is not None and x[3] > seuil_latence)
print(f"le nombre de requetes ayant abouties et dont le temps de latence à dépassé {seuil_latence}ms est: ",anomalies.count())
print("les ttl correspondant sont :", anomalies.map(lambda x: x[2]).distinct().collect())
ttl_rdd =anomalies
ttl_counts = ttl_rdd.map(lambda x: x[2]).countByValue()

print("Nombre d'occurrences pour chaque TTL:")
for ttl, count in ttl_counts.items():
    print(f"TTL {ttl}: {count}")

le nombre de requetes ayant abouties et dont le temps de latence à dépassé 15000ms est:  57
les ttl correspondant sont : [114, 105]
Nombre d'occurrences pour chaque TTL:
TTL 105: 26
TTL 114: 31


###### **d.  par exemple, 20 00 ms**

In [32]:
seuil_latence = 20000
anomalies = rdd_extrait.filter(lambda x: x[3] is not None and x[3] > seuil_latence)
print(f"le nombre de requetes ayant abouties et dont le temps de latence à dépassé {seuil_latence}ms est: ",anomalies.count())
print("les ttl correspondant sont :", anomalies.map(lambda x: x[2]).distinct().collect())
ttl_rdd =anomalies
ttl_counts = ttl_rdd.map(lambda x: x[2]).countByValue()

print("Nombre d'occurrences pour chaque TTL:")
for ttl, count in ttl_counts.items():
    print(f"TTL {ttl}: {count}")

le nombre de requetes ayant abouties et dont le temps de latence à dépassé 20000ms est:  30
les ttl correspondant sont : [114]
Nombre d'occurrences pour chaque TTL:
TTL 114: 30


##### **10.  le nombre d’occurrences où la latence est anormalement élevée**

In [33]:
import numpy as np
latences_rdd = rdd_extrait.filter(lambda x: x[3] is not None)
latences=latences_rdd.map(lambda x: x[3]).collect()
q1 = np.percentile(latences, 25)
q3 = np.percentile(latences, 75)
iqr = q3 - q1
borne_sup = q3 + 1.5 * iqr
valeurs_aberrantes = latences_rdd.filter(lambda x:  x[3] > borne_sup)

print(f"Le nombre d'occurrences où la latence est anormalement élevée (valeurs aberrantes) est: {valeurs_aberrantes.count()}")

Le nombre d'occurrences où la latence est anormalement élevée (valeurs aberrantes) est: 9685


##### **11.  le nombre d’occurrences où la latence est anormalement basse**

In [34]:
borne_inf = q1 - 1.5 * iqr
valeurs_aberrantes = latences_rdd.filter(lambda x:  x[3] < borne_inf)

print(f"Le nombre d'occurrences où la latence est anormalement basse (valeurs aberrantes) est: {valeurs_aberrantes.count()}")

Le nombre d'occurrences où la latence est anormalement basse (valeurs aberrantes) est: 71


In [38]:
borne_inf

np.float64(117.40687499999997)

##### **12.   Fréquences d'apparition des ttl**

In [35]:
ttl_rdd = rdd_extrait.filter(lambda x: x[2] is not None)    
ttl_counts = ttl_rdd.map(lambda x: x[2]).countByValue()

print("Nombre d'occurrences pour chaque TTL:")
for ttl, count in ttl_counts.items():
    print(f"TTL {ttl}: {count}")


Nombre d'occurrences pour chaque TTL:
TTL 105: 7810
TTL 100: 61
TTL 103: 190
TTL 114: 52883
TTL 110: 730
TTL 109: 4048


##### **12.    Variation moyenne  du délai de transmission des paquets ou gigue**

## Formule de la gigue(ou jitter en anglais)


La variation moyenne entre les valeurs de RTT successives est donnée par :

$$
Jitter = \frac{\sum |RTT_i - RTT_{i-1}|}{N-1}
$$

où \( RTT_i \) (Round-Trip Time) est le temps qu'un paquet met pour aller d'un émetteur à un récepteur et revenir lors de la \( i \)-ème requête envoyée, et \( N \) est le nombre total de requêtes.

Ce code extrait les valeurs RTT d’un RDD, crée des paires successives \( (RTT_i, RTT_{i-1}) \) via un décalage, filtre les `None`, puis calcule la moyenne des différences absolues pour obtenir la gigue (jitter).

In [36]:
# Créer un RDD contenant les valeurs RTT
rtt_rdd = rdd_extrait.map(lambda x: x[3])

# Créer un RDD décalé contenant les valeurs RTT décalées d'une position
rtt_decalé_rdd = rtt_rdd.zipWithIndex().map(lambda x: (x[1], x[0]))
rtt_original_rdd = rtt_rdd.zipWithIndex().map(lambda x: (x[1] + 1, x[0]))

# Joindre les deux RDD pour obtenir les paires de valeurs RTT successives
rtt_pairs_rdd = rtt_original_rdd.join(rtt_decalé_rdd).map(lambda x: x[1])

# Filtrer les paires où l'une des valeurs est None
rtt_valid_pairs_rdd = rtt_pairs_rdd.filter(lambda x: x[0] is not None and x[1] is not None)

# Calculer la différence absolue puis la moyenne
rtt_diff_rdd = rtt_valid_pairs_rdd.map(lambda x: abs(x[0] - x[1]))

jitter = rtt_diff_rdd.mean()

print(f"La gigue est : {jitter:.3f} ms")

La gigue est : 500.122 ms


D'après https://www.iptis.fr/blog/comprendre-la-qualite-de-sa-connexion-internet, une bonne gigue est d'une valeur inférieur à 20ms, ici la gigne est de 500.122, ce qui est largement supérieur à 20ms, on peut donc dire que le délai entre les paquets de données varie de manière significative, une gigue de 500.122 ms pourrait entraîner des délais importants avant de recevoir une réponse. Cela montre que la navigation sur internet peut devenir très lente, ceci à cause d'une grande fluctuation dans le temps que met chaque paquet pour parcourir le réseau, avec des variations allant de 0 à 500 ms

### **II.3. VISUALISATIONS**

In [46]:
import pandas as pd
import plotly.express as px

import plotly.io as pio
pio.renderers.default = "iframe"


In [47]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot


init_notebook_mode(connected=True)

In [43]:
received = rdd_extrait.filter(lambda x: x[3] is not None).count()

In [ ]:
from IPython.display import Image
import plotly.express as px
import plotly.graph_objects as go


labels = ['Reçues', 'Perdues']
sizes = [received, total - received]
colors = ['green', 'red']

fig = go.Figure(data=[go.Pie(labels=labels, values=sizes, hole=.3, 
                             marker=dict(colors=colors), 
                             textinfo='label+percent', 
                             pull=[0, 0.1])])
fig.update_layout(title_text='Taux de perte de paquets')
fig.show()



In [46]:
fig = px.box(latences, title='Box Plot des Latences')
fig.update_layout(xaxis_title='Latences', yaxis_title='Valeurs')
fig.show()

In [68]:
def moyenne_par_groupe(rdd_grouped):
    def moyenne(groupe):
        valeurs = [x[3] for x in groupe if x[3] is not None]
        somme = sum(valeurs)
        nombre = len(valeurs)
        return f"{(somme / nombre):.2f}" if nombre > 0 else None

    return rdd_grouped.map(lambda groupe: (groupe[0], moyenne(groupe[1])))

In [48]:
import pandas as pd
rdd_grp = rdd_indexed.map(lambda x: (x[1] // 10, x[0])) \
                 .groupByKey() \
                 .mapValues(list) \
                 .sortByKey()

moy_grp = moyenne_par_groupe(rdd_grp)

df_moy = pd.DataFrame(moy_grp.collect(), columns=['Groupe', 'Moyenne'])
df_moy['Moyenne'] = df_moy['Moyenne'].astype(float)

fig = px.line(df_moy, x='Groupe', y='Moyenne', title='Moyenne par groupe de 10s')
fig.update_layout(xaxis_title='Groupe', yaxis_title='Temps moyen (ms)')
fig.show()


In [48]:
rdd_grp = rdd_indexed.map(lambda x: (x[1] // 100, x[0])) \
                 .groupByKey() \
                 .mapValues(list) \
                 .sortByKey()

moy_grp = moyenne_par_groupe(rdd_grp)

df_moy = pd.DataFrame(moy_grp.collect(), columns=['Groupe', 'Moyenne'])
df_moy['Moyenne'] = df_moy['Moyenne'].astype(float)

fig = px.line(df_moy, x='Groupe', y='Moyenne', title='Moyenne par groupe de 100s')
fig.update_layout(xaxis_title='Groupe', yaxis_title='Temps moyen (ms)')
fig.show()


In [49]:
rdd_grp = rdd_indexed.map(lambda x: (x[1] // 300, x[0])) \
                 .groupByKey() \
                 .mapValues(list) \
                 .sortByKey()

moy_grp = moyenne_par_groupe(rdd_grp)

df_moy = pd.DataFrame(moy_grp.collect(), columns=['Groupe', 'Moyenne'])
df_moy['Moyenne'] = df_moy['Moyenne'].astype(float)

fig = px.line(df_moy, x='Groupe', y='Moyenne', title='Moyenne par groupe de 300s')
fig.update_layout(xaxis_title='Groupe', yaxis_title='Temps moyen (ms)')
fig.show()


In [ ]:
import plotly.express as px
ttl_values = rdd_received.map(lambda x: x[2]).collect()
fig = px.histogram(ttl_values, nbins=30, title='Distribution des TTL')
fig.update_layout(xaxis_title='TTL', yaxis_title='Effectif')
fig.show()


In [43]:
import numpy as np
import plotly.graph_objects as go


seuils = ["500ms", "5000ms", "15000ms", "20000ms"]
ttl_labels = ["100", "103", "105", "109", "110", "114"]


ttl_data = {
    "500ms": [5, 71, 2927, 98, 5, 151],
    "5000ms": [0, 0, 1100, 11, 0, 33],
    "15000ms": [0, 0, 26, 0, 0, 31],
    "20000ms": [0, 0, 0, 0, 0, 30],
}


ttl_values = np.array([ttl_data[seuil] for seuil in seuils]).T


ttl_totals = ttl_values.sum(axis=0)
ttl_values_normalized = ttl_values / ttl_totals  


fig = go.Figure()


for i, ttl in enumerate(ttl_labels):
    fig.add_trace(go.Bar(
        x=seuils,
        y=ttl_values_normalized[i],
        name=f"TTL {ttl}",
        text=[f"{value*100:.1f}%" for value in ttl_values_normalized[i]],  # Affichage des pourcentages
        textposition='inside',
        hoverinfo='x+y+name',
    ))


fig.update_layout(
    barmode='stack',  
    title="Distribution des TTL pour chaque seuil de latence (proportions)",
    xaxis_title="Seuils de latence",
    yaxis_title="Proportion (%)",
    legend_title="TTL",
    template="plotly_white",
)

# Affichage du graphique
fig.show()
